In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import os
from utils import utils
import re

os.environ["CUDA_VISIBLE_DEVICES"] = "9"

INFO 05-05 22:00:37 __init__.py:190] Automatically detected platform cuda.


In [2]:
model_name = "./data/models/Qwen3-8B"
model = LLM(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

INFO 05-05 22:00:48 config.py:542] This model supports multiple tasks: {'generate', 'classify', 'reward', 'embed', 'score'}. Defaulting to 'generate'.
WARNING 05-05 22:00:48 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 05-05 22:00:48 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 05-05 22:00:48 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='./data/models/Qwen3-8B', speculative_config=None, tokenizer='./data/models/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 05-05 22:00:54 model_runner.py:1115] Loading model weights took 15.2585 GB
INFO 05-05 22:00:55 worker.py:267] Memory profiling takes 0.77 seconds
INFO 05-05 22:00:55 worker.py:267] the current vLLM instance can use total_gpu_memory (47.41GiB) x gpu_memory_utilization (0.90) = 42.67GiB
INFO 05-05 22:00:55 worker.py:267] model weights take 15.26GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 25.95GiB.
INFO 05-05 22:00:55 executor_base.py:110] # CUDA blocks: 11811, # CPU blocks: 1820
INFO 05-05 22:00:55 executor_base.py:115] Maximum concurrency for 40960 tokens per request: 4.61x
INFO 05-05 22:00:56 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.22it/s]

INFO 05-05 22:01:12 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.71 GiB
INFO 05-05 22:01:12 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 18.35 seconds


In [3]:
math_500 = utils.read_from_json("./data/proved_Math-500.json")

In [7]:
test_str = math_500[0]["generated_proof_results"][0]["response_proof_generation"]

In [11]:
import re
match = re.search(r'<think>(.*?)(</think>|$)', test_str, re.DOTALL)
if match:
    long_cot_content = match.group(1)

In [10]:
math_500[0].keys()

dict_keys(['problem', 'solution', 'answer', 'subject', 'level', 'unique_id', 'Name', 'translation_generated_results', 'autoformalization_generated_results', 'generated_proof_results'])

In [18]:
prompt = """Find the answer to the following question in the provided long CoT content. Your answer should be in \\boxed\{\} format.

Here is the question:
""" + math_500[0]["problem"] + f"""

Here is the long CoT content:
{long_cot_content}"""

messages = [
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True, 
    enable_thinking=False
)

print(text)

<|im_start|>user
Find the answer to the following question in the provided long CoT content. Your answer should be in \boxed\{\} format.

Here is the question:
Compute $58_9 - 18_9.$ Express your answer in base $9.$

Here is the long CoT content:

# Solving the Base 9 Subtraction Problem in Lean 4

We need to compute $58_9 - 18_9$ and express the answer in base 9. Let's first convert these numbers to base 10 to perform the subtraction, and then convert back to base 9.

The number $58_9$ in base 10 is $5 \cdot 9^1 + 8 \cdot 9^0 = 5 \cdot 9 + 8 \cdot 1 = 45 + 8 = 53$.

The number $18_9$ in base 10 is $1 \cdot 9^1 + 8 \cdot 9^0 = 1 \cdot 9 + 8 \cdot 1 = 9 + 8 = 17$.

Subtracting these gives $53 - 17 = 36$.

Now we need to convert $36_{10}$ back to base 9:
- $36 \div 9 = 4$ remainder $0$
- $4 \div 9 = 0$ remainder $4$

So $36_{10} = 40_9$.

Let's translate this into Lean 4:

```tactics
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat
```

Fir

In [19]:
sampling_params = SamplingParams(temperature=0.6, top_p=0.95, max_tokens=8096)
output = model.generate(text, sampling_params=sampling_params)
output_text = output[0].outputs[0].text
print(output_text)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.44s/it, est. speed input: 251.27 toks/s, output: 34.17 toks/s]

The question asks to compute $58_9 - 18_9$ and express the answer in base 9.

First, we convert both numbers from base 9 to base 10:
- $58_9 = 5 \cdot 9^1 + 8 \cdot 9^0 = 45 + 8 = 53$
- $18_9 = 1 \cdot 9^1 + 8 \cdot 9^0 = 9 + 8 = 17$

Next, we subtract the base 10 values:
$$
53 - 17 = 36
$$

Now, we convert $36_{10}$ back to base 9:
- $36 \div 9 = 4$ remainder $0$
- $4 \div 9 = 0$ remainder $4$

Thus, $36_{10} = 40_9$.

Therefore, the answer is:

$$
\boxed{40}
$$


In [14]:
matches = re.findall(r'\\boxed\{(.*?)\}', output_text)

if matches:
    last_boxed = matches[-1]
    print(last_boxed)

40


In [16]:
math_500[0]["answer"]

'40_9'